# Machine Learning on top 6 measurements
## Missing data is filled in with mean values of each column

In [1]:
# Dependencies
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import pickle
from sklearn.ensemble import GradientBoostingClassifier
# ***Everything below not required for gradient boosting model***
# For deep learning model
from sklearn.preprocessing import LabelEncoder, StandardScaler
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.models import load_model

C:\Users\Al\Anaconda3\envs\PythonData\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
neuro = pd.read_csv("all_neuron_data.csv")

In [3]:
# Filter down our dataset to only use top 6 measurements
neuro_filtered = neuro.loc[(neuro["Measurement"] == "input resistance") |
                          (neuro["Measurement"] == "resting membrane potential") |
                          (neuro["Measurement"] == "spike threshold") |
                          (neuro["Measurement"] == "spike half-width") |
                          (neuro["Measurement"] == "spike amplitude") |
                          (neuro["Measurement"] == "membrane time constant")]

neuro_filtered.head()

,Cell Type,Value,Measurement
0,Dorsal root ganglion cell,-54.3,resting membrane potential
1,Dorsal root ganglion cell,-27.4,spike threshold
2,Dorsal root ganglion cell,101.3,spike amplitude
5,Dorsal root ganglion cell,192.0,input resistance
7,Spinal cord intermediate horn motor neuron sym...,-59.8,resting membrane potential


In [4]:
# Create a list of dictionaries where each dictionary item is a cell type with all of its firing characteristics grouped 
# together (unlike before where each entry is only a single measurement for a single cell)
current_cell = ""
current_dict = {}
big_list = []

for index, row in neuro_filtered.iterrows():
    if current_cell != row["Cell Type"]:
        # if the current cell and next cell are different, make a new empty row
        big_list.append(current_dict)
        current_dict = {}
        current_cell = row["Cell Type"]
        current_dict["Cell Type"] = current_cell
        current_dict[row["Measurement"]] = row["Value"]
    else:
        # else add the measurement value into the appropriate measurement column
        current_dict[row["Measurement"]] = row["Value"]

del big_list[0]
big_list[0]

{'Cell Type': 'Dorsal root ganglion cell',
 'input resistance': 192.0,
 'resting membrane potential': -54.3,
 'spike amplitude': 101.3,
 'spike threshold': -27.4}

In [5]:
# Fill in NaN values with mean values of that cell type's measurement (eg. the mean value of all values for a particular 
# cell's input resistance column will be filled in for that cell where values are missing in input resistance)

mean_clean = pd.DataFrame(big_list)
mean_clean["input resistance"] = mean_clean.groupby("Cell Type")["input resistance"].transform(lambda x: x.fillna(x.mean()))
mean_clean["membrane time constant"] = mean_clean.groupby("Cell Type")["membrane time constant"].transform(lambda x: x.fillna(x.mean()))
mean_clean["resting membrane potential"] = mean_clean.groupby("Cell Type")["resting membrane potential"].transform(lambda x: x.fillna(x.mean()))
mean_clean["spike amplitude"] = mean_clean.groupby("Cell Type")["spike amplitude"].transform(lambda x: x.fillna(x.mean()))
mean_clean["spike half-width"] = mean_clean.groupby("Cell Type")["spike half-width"].transform(lambda x: x.fillna(x.mean()))
mean_clean["spike threshold"] = mean_clean.groupby("Cell Type")["spike threshold"].transform(lambda x: x.fillna(x.mean()))

mean_clean = mean_clean.dropna()
mean_clean = mean_clean.loc[mean_clean["Cell Type"] != "Other"]
mean_clean.head()

,Cell Type,input resistance,membrane time constant,resting membrane potential,spike amplitude,spike half-width,spike threshold
0,Dorsal root ganglion cell,192.00,14.630,-54.30,101.300000,3.122222,-27.40
2,Hippocampus CA1 pyramidal cell,100.60,27.900,-64.80,84.865093,1.600000,-51.50
3,Cerebellar nucleus cell,59.26,58.175,-58.84,72.225000,2.130000,-68.58
4,Hippocampus CA3 pyramidal cell,164.00,61.000,-76.00,78.985714,0.790000,-58.00
5,Basalis nucleus cholinergic neuron,268.00,28.200,-48.00,66.700000,0.520000,-31.90


In [6]:
# Number of unique cell types in the dataset
mean_clean["Cell Type"].nunique()

65

In [7]:
# Full dictionary of cell types and number of occurrences
unique, counts = np.unique(mean_clean["Cell Type"], return_counts=True)
dict(zip(unique, counts))

{'Amygdala basolateral nucleus pyramidal neuron': 18,
 'Amygdala corticomedial nucleus pyramidal cell': 1,
 'BNST (ALG)': 8,
 'BNST common spiny neuron': 9,
 'Basalis nucleus cholinergic neuron': 15,
 'Cerebellar nucleus cell': 6,
 'Cerebellum granule cell': 14,
 'Dentate gyrus basket cell': 5,
 'Dentate gyrus granule cell': 57,
 'Dentate gyrus mossy cell': 6,
 'Dorsal root ganglion cell': 49,
 'Globus pallidus intrinsic cell': 2,
 'Hippocampus CA1 IS-I neuron': 10,
 'Hippocampus CA1 basket cell': 27,
 'Hippocampus CA1 ivy neuron': 1,
 'Hippocampus CA1 neurogliaform cell': 3,
 'Hippocampus CA1 oriens lacunosum moleculare neuron': 13,
 'Hippocampus CA1 pyramidal cell': 164,
 'Hippocampus CA1 radiatum giant cell': 3,
 'Hippocampus CA2 pyramidal neuron': 8,
 'Hippocampus CA3 lacunosum moleculare neuron': 7,
 'Hippocampus CA3 pyramidal cell': 49,
 'Hippocampus CA3 stratum radiatum giant cell': 5,
 'Hypoglossal nucleus motor neuron': 8,
 'Hypothalamus oxytocin neuroendocrine magnocellular c

In [8]:
# Obsolete, do not run (this fills in NA values of columns with the mean values and doesn't take into account cell type)
# cleaned = pd.DataFrame(big_list)

# mean_input_resist = cleaned["input resistance"].mean()
# mean_mem_const = cleaned["membrane time constant"].mean()
# mean_resting_mem = cleaned["resting membrane potential"].mean()
# mean_spike_amp = cleaned["spike amplitude"].mean()
# mean_spike_half = cleaned["spike half-width"].mean()
# mean_spike_thresh = cleaned["spike threshold"].mean()

# cleaned["input resistance"].fillna(mean_input_resist, inplace=True)
# cleaned["membrane time constant"].fillna(mean_mem_const, inplace=True)
# cleaned["resting membrane potential"].fillna(mean_resting_mem, inplace=True)
# cleaned["spike amplitude"].fillna(mean_spike_amp, inplace=True)
# cleaned["spike half-width"].fillna(mean_spike_half, inplace=True)
# cleaned["spike threshold"].fillna(mean_spike_thresh, inplace=True)

# cleaned

In [9]:
# cleaned = cleaned.loc[cleaned["Cell Type"] != "Other"]
# cleaned

In [10]:
# dropped = pd.DataFrame(big_list)
# dropped = dropped.dropna(axis=0)
# dropped.head()

In [11]:
# Run machine learning on this narrowed down dataset
X=mean_clean.drop("Cell Type", axis=1)
y=mean_clean["Cell Type"]

In [12]:
x_train, x_test, y_train, y_test = train_test_split(X,y,test_size=0.20, random_state=42)

In [13]:
mean_clean["spike threshold"].max()

763.0

In [14]:
x_test.head()

,input resistance,membrane time constant,resting membrane potential,spike amplitude,spike half-width,spike threshold
1658,458.000000,10.385714,-67.628364,65.474706,0.606828,-38.150520
1064,316.900000,44.600000,-75.100000,76.201111,1.398000,-41.764091
1553,150.641026,12.198462,-78.650000,65.502069,1.184231,-41.618276
705,164.947000,10.385714,-64.500000,65.474706,0.606828,-38.150520
685,262.159310,20.062941,-61.500000,70.283889,0.931931,-43.108421


### Random Forest model

In [27]:
#RFcls = Random Forest classifier
RFcls= RandomForestClassifier(n_estimators=500, random_state=42)
RFcls.fit(x_train, y_train)
y_predict = RFcls.predict(x_test)

In [28]:
# how many right divided by the total
print(accuracy_score(y_test, y_predict))

0.8481012658227848


In [29]:
# Save the random forest model in the model directory

filepath = '../model/machine_learning3_forest.sav'
pickle.dump(RFcls, open(filepath, 'wb'))

In [30]:
loaded_model = pickle.load(open(filepath, 'rb'))
result = loaded_model.score(x_test, y_test)
print(result)

0.8481012658227848


### Gradient Boosting model (takes forever to run)

In [17]:
#GBcls = Gradient Boosting classifier
GBcls= GradientBoostingClassifier(n_estimators=500, random_state=42)
GBcls.fit(x_train, y_train)
y_predict = GBcls.predict(x_test)

In [18]:
# Test accuracy of model
print(accuracy_score(y_test, y_predict))

0.8607594936708861


In [26]:
# Save the model in the model directory
filepath = '../model/machine_learning3_gradboost.sav'
pickle.dump(GBcls, open(filepath, 'wb'))

### Load boosting model, take in user input

In [21]:
# load model
filepath = '../model/machine_learning3_gradboost.sav'
gradboost_model = pickle.load(open(filepath, 'rb'))
result = gradboost_model.score(x_test, y_test)
print(result)

0.8607594936708861


In [22]:
# Pick any row from our train or test data as input
user_input = np.array(x_train.iloc[42]).reshape(1,-1)

In [23]:
# pd.DataFrame({"true": y_test, "pred": y_predict})

#predicts probabilities for that class
# round(gradboost_model.predict_proba(user_input).max()*100,2)

# jsonify and pass to front end
output = {"predictions": gradboost_model.predict(user_input)[0], "probability": round(gradboost_model.predict_proba(user_input).max()*100,2)}
#jsonify this output
output

{'predictions': 'Neocortex pyramidal cell layer 5-6', 'probability': 100.0}

In [24]:
# Validate that the prediction was actually correct
y_train.iloc[42]

'Neocortex pyramidal cell layer 5-6'

In [25]:
# Number of classifications
y.nunique()

65

### Deep learning model

In [ ]:
y_predict

In [ ]:
# Machine learning model will not work on data categories with only 1 row
# dropping all rows that have fewer than 5 data points associated with a category

counts = mean_clean['Cell Type'].value_counts()
greater_than_five = mean_clean[mean_clean['Cell Type'].isin(counts[counts >= 40].index)]

# Run machine learning on this narrowed down dataset
X=greater_than_five.drop("Cell Type", axis=1)
y=greater_than_five["Cell Type"]

In [ ]:
greater_than_five

In [ ]:
greater_than_five["Cell Type"].value_counts()

In [ ]:
# This code works in machine_learning2 notebook but not here. y is the same.  How come?
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [ ]:
X_scaler = StandardScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

# Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [ ]:
y_train_list = y_train.tolist()

for i in range(0,len(y_train_list)):
    print(y_train_list[i] + " is now " + str(y_train_categorical[i]))

In [ ]:
# Create model and add layers
model = Sequential()
model.add(Dense(units=18, activation='relu', input_dim=6))
model.add(Dense(units=20, activation='relu'))
model.add(Dense(units=11, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=100,
    shuffle=True,
    verbose=2
)

In [ ]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
# Save the model
model.save("machine_learning3_seq.h5")

# Getting model to predict a neuron (user input is hard-coded)

In [ ]:
data = np.array([192,14,-54,101,3,-27]).reshape(6,)
# print(data.shape)
prediction = model.predict(np.array([data]))
print(prediction.shape)
# prediction=prediction.astype(int)
# prediction_string = prediction.argmax(axis=-1)
# type(prediction_string)
print(prediction)
# print(list(label_encoder.classes_))
# list(label_encoder.inverse_transform(prediction))

# prediction_string = keras.np_utils.probas_to_classes(prediction)
# prediction_string
# prediction_string = model.predict_classes(np.array([data]))
# prediction_string
print(type(prediction))

In [ ]:
print(prediction.tolist())
the_pred = prediction.tolist()
flat_pred= [y for x in the_pred for y in x]
print(flat_pred)
count = 0
for i in flat_pred:
    count +=1
    #print(i)
    #print(flat_pred[i])
    if i == 1:
        the_index = i
        print(label_encoder.inverse_transform(count))

In [ ]:
y_train_list = y_train.tolist()

for i in range(0,len(y_train_list)):
    print(y_train_list[i] + " is now " + str(y_train_categorical[i]))

In [ ]:
prediction
np.unique(y_train_categorical)
type(y_train_categorical)
unique_rows = np.unique(y_train_categorical, axis=0)
unique_rows

In [ ]:
# I can't zip this together with unique_rows because they're not in the same order
my_set = set(y_train_list)
my_set

In [ ]:
y_train_categorical

In [ ]:
np.array(y_train)

In [ ]:
list(prediction)

In [ ]:
# zip pre-encoded and post-encoded classifications together
categories = list(zip(y_train_categorical, np.array(y_train)))
categories[1]

my_lists = []
for i in categories:
    my_lists.append(list(i))
my_lists[1]

In [ ]:
# if the prediction array is the same as the encoded classification, print the corresponding label
for i in my_lists:
#     print(i[0])
#     print(prediction[0])
    if all(i[0] == prediction[0]):
        result = i[1]
print(result)

In [ ]:
y_train_categorical

# Load model, predict using dynamic user data

In [ ]:
seq_model=load_model("machine_learning3_seq.h5")

In [ ]:
# Saving minimum and maximum values for each measurement into variables

# Min/ max input resistance values
min_ir = greater_than_five["input resistance"].min()
max_ir = greater_than_five["input resistance"].max()
# Min/max membrane time constant values
min_mtc = greater_than_five["membrane time constant"].min()
max_mtc = greater_than_five["membrane time constant"].max()
# Min/ max resting membrane potential values
min_rmp = greater_than_five["resting membrane potential"].min()
max_rmp = greater_than_five["resting membrane potential"].max()
# Min/max spike amplitude values
min_sa = greater_than_five["spike amplitude"].min()
max_sa = greater_than_five["spike amplitude"].max()
# Min/max spike half-width values
min_shw = greater_than_five["spike half-width"].min()
max_shw = greater_than_five["spike half-width"].max()
# Min/max spike threshold values
min_st = greater_than_five["spike threshold"].min()
max_st = greater_than_five["spike threshold"].max()

In [ ]:
# Attempt to pass in user input into "data" below using input()
input_resistance = input("input_resistance value (between {} and {}): ".format(min_ir, max_ir))
membrane_time_constant = input("membrane_time_constant value (between {} and {}): ".format(min_mtc, max_mtc))
resting_membrane_potential = input("resting_membrane_potential value (between {} and {}): ".format(min_rmp, max_rmp))
spike_amplitude = input("spike_amplitude value (between {} and {}): ".format(min_sa, max_sa))
spike_halfwidth = input("spike_half-width value (between {} and {}): ".format(min_shw, max_shw))
spike_threshold = input("spike_threshold value (between {} and {}): ".format(min_st, max_st))

input_resistance
membrane_time_constant
resting_membrane_potential
spike_amplitude
spike_halfwidth
spike_threshold

In [ ]:
# Place user data into numpy array in same order as dataframe's columns
user_data = np.array([input_resistance, membrane_time_constant, resting_membrane_potential, spike_amplitude, spike_halfwidth, spike_threshold]).reshape(6,).astype(np.float32)
user_data

In [ ]:
prediction = seq_model.predict(np.array([user_data]))
prediction

In [ ]:
categories = list(zip(y_train_categorical, np.array(y_train)))
categories[1]

my_lists = []
for i in categories:
    my_lists.append(list(i))
my_lists[1]

In [ ]:
y_train.unique()

In [ ]:
for i in my_lists:
    print(i[1])

In [ ]:
for i in my_lists:
#     print(i[0])
#     print(prediction[0])
    if all(i[0] == prediction[0]):
        result = i[1]
print(result + )

# Filter data, run machine learning on filtered data (probably won't use this)

In [ ]:
words = ["CA1", "CA3"]
for i, row in cleaned.iterrows():
    if any(word in row["Cell Type"] for word in words):
#         print("true")0
        cleaned.loc[i, "Area"] = "Hippocampus"
#         row["Area"] = "Hippocampus"
    else:
        cleaned.loc[i, "Area"] = "other"
        row["Area"] = "other"

In [ ]:
hippo_df = cleaned.loc[cleaned["Area"] == "Hippocampus"]
X=hippo_df.drop(["Cell Type", "Area"], axis=1)
y=hippo_df["Cell Type"]

In [ ]:
print(y)

In [ ]:
y=y.tolist()

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

X_scaler = StandardScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Step 1: Label-encode data set
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

# Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [ ]:
y_train

In [ ]:
y_train_categorical